# Libraries

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras import utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

# Get data

In [3]:
a = pd.read_csv('iris.csv', header = None)
i = pd.DataFrame(a)
iris = i.values

X = iris[:, :4].astype(float)  # Features
Y = iris[:, 4]  # Labels

In [4]:
# Set Seed
seed = 7
np.random.seed(seed)

In [5]:
# Label encode Class (Species)
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# One Hot Encode
y_dummy = utils.to_categorical(encoded_Y)

In [6]:
# Deep Learnig Function
def deepml_model():
    # Model Creation
    deepml = Sequential()
    deepml.add(Dense(8, input_dim=4, activation='relu'))
    deepml.add(Dense(3, activation='softmax'))
    # Model Compilation
    deepml.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return deepml

# 10-fold

In [7]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

acc_scores = []

for train, test in k_fold.split(X, encoded_Y):
    # Create model
    model = deepml_model()
    
    # Train model
    model.fit(X[train], y_dummy[train], epochs=200, batch_size=5, verbose=0)
    
    # Evaluate model
    scores = model.evaluate(X[test], y_dummy[test], verbose=0)
    print(f'Score for fold: {scores[1]*100}%')
    acc_scores.append(scores[1] * 100)

# Calculate mean and standard deviation of scores
mean_acc = np.mean(acc_scores)
std_acc = np.std(acc_scores)
print(f'Model: {mean_acc:.2f}% ({std_acc:.2f}%)')

c:\Users\ICojocaru\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Score for fold: 93.33333373069763%
Score for fold: 93.33333373069763%
Score for fold: 100.0%
Score for fold: 100.0%
Score for fold: 100.0%
Score for fold: 93.33333373069763%
Score for fold: 93.33333373069763%
Score for fold: 100.0%
Score for fold: 100.0%
Score for fold: 100.0%
Model: 97.33% (3.27%)


# Ensemble

In [8]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_dummy, test_size=0.2, random_state=seed)

# Train an ensemble of 10 models
models = [deepml_model() for _ in range(10)]
for model in models:
    model.fit(X_train, y_train, epochs=200, batch_size=5, verbose=0)

# Predict with the ensemble and average the predictions
predictions = [model.predict(X_test) for model in models]
predictions = np.mean(predictions, axis=0)
ensemble_pred = np.argmax(predictions, axis=1)

# Evaluate the ensemble
test_true = np.argmax(y_test, axis=1)
accuracy = accuracy_score(test_true, ensemble_pred)
print("Ensemble Accuracy: {:.2f}%".format(accuracy * 100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Ensemble Accuracy: 93.33%
